# Sequências - Aula Prática
## Embeddings

Neste notebook iremos trabalhar um pouco com a parte introdutória de modelos de linguagem, realizando **analogias** a partir de representações distribuídas (*embeddings*) geradas pelo algoritmo de aprendizagem não-supervisionada `GloVe` (*Global Vectors for Word Representation*).
> Para saber mais sobre como o `GloVe` funciona, recomendo a leitura do seguinte [notebook](https://colab.research.google.com/github/jaygala24/pytorch-implementations/blob/master/Global%20Vectors%20for%20Word%20Representation.ipynb#scrollTo=oEXgG-hDIMdT). Esse notebook prático não tem como foco explicar como que o algoritmo funciona, mas sim explicar as operações que podemos fazer com as representações distribuídas geradas por algoritmos de *word embedding*.

- Este notebook foi inspirado pelos trabalhos disponibilizados nos sites [d2l.ai](https://d2l.ai/chapter_natural-language-processing-pretraining/similarity-analogy.html) e [notebook.community](https://notebook.community/spro/practical-pytorch/glove-word-vectors/glove-word-vectors).

## Importação de pacotes

In [ ]:
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

from os.path import join as ospj
from torch.utils.data import Dataset

sns.set_style('dark')

## Analogias

Analogias são associações de mesma natureza entre palavras (como flexões de gênero ou número). A geometria dessas associações pode ser visualizada no espaço vetorial onde as palavras são projetadas e, em modelos bem treinados, deve ser possível encontrar semelhanças entre associações de mesma natureza.

<img width=800 src="https://github.com/ThiagoPoppe/ciag2024/blob/main/imagens/analogias.png?raw=true"/>

Ao longo deste notebook, utilizaremos o modelo `GloVe` para realizar a projeção de palavras em um espaço vetorial. Mais especificamente, iremos carregar um modelo pré-treinado em uma base de dados de 6 bilhões de tokens, realizando uma projeção das palavras em um espaço 100 dimensional.

### Carregamento de um modelo GloVe pré-treinado

Iremos fazer o download de um modelo `GloVe` pré-treinado. A classe `GloVe` serve para representar o modelo.

> O modelo possui alguns atributos, como o dicionário `stoi` (*string* to *int*) para mapear uma palavra para um índice numérico e a lista `itos` (*int* to *string*), mapeando um índice númerico para uma palavra. Além disso, conseguimos acessar a matriz de *embeddings* através do atributo `vectors`.

In [ ]:
class GloVe:
    def __init__(self):
        self.itos = []
        self.stoi = {}

        vectors = []

        with open('/pgeoprj2/ciag2024/dados/glove.6B.100d.txt', encoding="utf8") as f:
            for i, line in enumerate(f):
                word, *vector = line.split()
                vectors.append(np.array(vector, dtype=np.float32)[:100])
                self.itos.append(word)
                self.stoi[word] = i

        self.vectors = torch.tensor(np.array(vectors))

    def get_vecs_by_tokens(self, tokens):
        embeddings = []
        to_reduce = False

        if not isinstance(tokens, list):
            tokens = [tokens]
            to_reduce = True

        for token in tokens:
            if token in self.stoi:
                idx = self.stoi[token]
                embeddings.append(self.vectors[idx])
            else:
                embeddings.append(torch.zeros(100))

        vecs = torch.stack(embeddings)
        return vecs[0] if to_reduce else vecs

In [ ]:
glove = GloVe()

random_tokens = np.random.choice(glove.itos, size=5)

print('Número de tokens mapeados pelo glove.6B.100d:', len(glove.stoi))
print('Aqui temos 5 tokens aleatórios mapeados pelo glove.6B.100d:', random_tokens)
print('A dimensionalidade da matriz de embeddings é:', glove.vectors.shape)

### Projeções vetoriais utilizando o GloVe

Podemos realizar projeções de palavras utilizando a matriz de *embeddings* computada pelo `glove.6B.100d` através do método `get_vecs_by_tokens`, como visto a seguir.

In [ ]:
tokens = ['man', 'king', 'woman', 'queen']
vecs = glove.get_vecs_by_tokens(tokens)

plt.matshow(vecs, aspect='auto')
plt.yticks(range(4), tokens)
plt.colorbar()
plt.show()

Percebemos que o GloVe também possui alguns *tokens* bem específicos sobre algumas áreas do conhecimento, como a palavra **maastrichtian** (geologia).

In [ ]:
glove.get_vecs_by_tokens(['maastrichtian'])

Caso o GloVe não tenha um *token* mapeado, não encontramos nenhum erro, apenas um *embedding* de zeros.
> Talvez o *token* **pre-salt** possa aparecer em modelos maiores: como o `glove.840B.300d`, por ter sido treinado num *corpus* bem maior (840 bilhões de *tokens*) do que o que estamos utilizando nesse notebook.

In [ ]:
glove.get_vecs_by_tokens(['pre-salt'])

Além disso, podemos visualizar esses vetores no espaço latente para analisar algumas propriedades interessantes. Para tornar a visualização factível, podemos utilizar um algoritmo de redução de dimensionalidade.

> Para esse exemplo iremos utilizar o algoritmo de redução de dimensionalidade PCA (*Principal Components Analysis*), devido a uma melhor interpretabilidade para o exemplo escolhido. Porém, é mais comum vermos algoritmos como o t-SNE (*t-distributed Stochastic Neighbor Embedding*), uma vez que este algoritmo preserva a estrutura local do dado original. Para mais detalhes na diferença entre t-SNE e PCA, recomendo a leitura do seguinte [link](https://medium.com/analytics-vidhya/pca-vs-t-sne-17bcd882bf3d).

In [ ]:
from sklearn.decomposition import PCA

tokens = ['man', 'woman', 'king', 'queen', 'brother', 'sister']
vecs = glove.get_vecs_by_tokens(tokens)

# Calculando a redução de dimensionalidade para um espaço bidimensional
pca = PCA(n_components=2)
components = pca.fit_transform(vecs.numpy())

print('Tamanho dos componentes:', components.shape)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(components[:, 0], components[:, 1])

for i, token in enumerate(tokens):
    ax.annotate(token, (components[i, 0], components[i, 1]))

for i in range(0, len(tokens) - 1, 2):
    x = (components[i, 0], components[i+1, 0])
    y = (components[i, 1], components[i+1, 1])
    ax.plot(x, y, linestyle='--')

Note pela imagem acima como a operação vetorial: $(\overrightarrow{\text{king}} - \overrightarrow{\text{queen}}) + \overrightarrow{\text{sister}} \approx \overrightarrow{\text{brother}}$.

In [ ]:
king = components[tokens.index('king')]
queen = components[tokens.index('queen')]
sister = components[tokens.index('sister')]
brother = components[tokens.index('brother')]

result = (king - queen) + sister
print('Vetor resultante:', result)
print('Vetor para a palavra "brother":', brother)

<br> Podemos refazer o plot para verificar onde a posição do vetor resultante.

In [ ]:
fig, ax = plt.subplots()

# Adicionando o ponto do vetor resultante
ax.scatter(result[0], result[1], color='red')
ax.annotate('resultante', (result[0], result[1]))

ax.scatter(components[:, 0], components[:, 1])

for i, token in enumerate(tokens):
    ax.annotate(token, (components[i, 0], components[i, 1]))

for i in range(0, len(tokens) - 1, 2):
    x = (components[i, 0], components[i+1, 0])
    y = (components[i, 1], components[i+1, 1])
    ax.plot(x, y, linestyle='--')

### Realizando analogias através do GloVe

Através de operações vetoriais, como visto anteriormente, somos capazes de criar analogias no espaço latente da nossa representação distribuída, conseguindo responder perguntas como: `rainha está para rei assim como irmã está para ???`.

Possuímos diversas estratégias para retornar a palavra correta para tal pergunta. Por exemplo, a partir da imagem da célula anterior, uma possível estratégia é de calcular o top-1 vizinho mais próximo do vetor resultante através de um algoritmo de `KNN`, porém tal estratégia pode ser menos viável em dimensões maiores devido à [maldição da dimensionalidade](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6065061). Para esse notebook, iremos utilizar uma estratégia que envolve a similaridade de cosseno entre os vetores no espaço latente, selecionando como analogia aquele vetor com maior similaridade.

A similaridade de cosseno pode ser computada através da seguinte equação, onde temos uma operação de similaridade no numerador (produto interno), dividido pelo produto da norma de cada vetor. Também introduzimos um fator $\epsilon \approx 1\mathrm{e}{-6}$ no denominador como um termo para evitar divisão por zero. A distância, ou similaridade, de cosseno possui imagem entre $[-1, +1]$, onde $-1$ indica que os vetores estão apontados para sentidos opostos; $+1$ indica que os vetores estão apontando para a mesma direção; e $0$ indica que os vetores são perpendiculares.

$$
\text{similaridade} = \frac{\overrightarrow{x_1} \cdot \overrightarrow{x_2}}{\max(\lVert \overrightarrow{x_1} \rVert_2 * \lVert \overrightarrow{x_2} \rVert_2, \epsilon)}
$$

Primeiramente, iremos observar os top vetores retornados pela distância de cosseno a partir de uma *query* (palavra) fornecida.

1. Implemente a função `get_similar_tokes`. Ela deve retornar um array com os tokens mais próximos de *query*.

In [ ]:
def get_similar_tokens(query, embedding, k=10):
    # Implemente a sua solução aqui

similar_tokens = get_similar_tokens('computer', glove, k=5)
for i, (token, similarity) in enumerate(similar_tokens):
    print(f'{i+1}. {token} -> {similarity:.5f}')

<br> Agora iremos utilizar a distância de cosseno para computar analogias.

2. Implemente a função `get_topk_analogies`.

In [ ]:
def get_topk_analogies(token1, token2, token3, embedding, k=5):
    # Implemente a sua solução aqui

get_topk_analogies('queen', 'king', 'wife', glove)

In [ ]:
get_topk_analogies('beijing', 'china', 'tokyo', glove)

In [ ]:
get_topk_analogies('do', 'does', 'go', glove)

## *Fine-tuning* de *embeddings* pré-treinados

No mundo de processamento de linguagem natural, é muito comum utilizarmos *embeddings* pré-treinados como um pontapé inicial para a nossa rede, muito por conta desses *embeddings* serem treinados em uma base de dados imensa, necessitando de muito poder computacional se quisermos treinar os nossos próprios *embeddings* do zero.

Caso você queira utilizar os pesos pré-treinados do `GloVe`, ou de qualquer outro modelo pré-treinado, em uma camada `nn.Embedding` do seu modelo, você pode inicializar os pesos dessa camada através da função `.from_pretrained`, como ilustrado a seguir.

In [ ]:
my_embedding = nn.Embedding.from_pretrained(glove.vectors)
my_embedding.weight

<br>Também temos a opção de manualmente dizer que queremos que esses pesos sofram um *fine-tuning* ao longo do treinamento.

> Por padrão, os pesos do modelo pré-treinado são "congelados", indicando que eles não sofrerão atualizações via *backpropagation*.

In [ ]:
my_embedding = nn.Embedding.from_pretrained(glove.vectors, freeze=False)
my_embedding.weight